## Using Opensearch Python Client

In [1]:
from opensearchpy import OpenSearch

In [12]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin')

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)


In [51]:
import json
def print_json(x):
    print(json.dumps(x, indent=2))

In [43]:
index_name = 'python-test-index'
index_body = {    
    'settings': {        
        'index': {            
            'number_of_shards': 2        
         }    
    }
}

In [44]:
response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print_json(response)


Creating index:
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "python-test-index"
}


In [45]:
document = {    
    'title': 'Moneyball',    
    'director': 'Bennett Miller',    
    'year': '2011'
}
id = '1'
 
response = client.index(    
    index = index_name,
    body = document,
    id = id,
    refresh = True
)
 
print('\nAdding document:')
print_json(response)


Adding document:
{
  "_index": "python-test-index",
  "_id": "1",
  "_version": 1,
  "result": "created",
  "forced_refresh": true,
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 0,
  "_primary_term": 1
}


In [46]:
q = 'miller'
query = {
    'size': 5,
    'query': {
        'multi_match': {
            'query': q,
            'fields': ['title^2', 'director']
        }
    }
}
 
response = client.search(
    body=query,
    index=index_name
)
print('\nSearch results:')
print_json(response)


Search results:
{
  "took": 2,
  "timed_out": false,
  "_shards": {
    "total": 2,
    "successful": 2,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1,
      "relation": "eq"
    },
    "max_score": 0.2876821,
    "hits": [
      {
        "_index": "python-test-index",
        "_id": "1",
        "_score": 0.2876821,
        "_source": {
          "title": "Moneyball",
          "director": "Bennett Miller",
          "year": "2011"
        }
      }
    ]
  }
}


In [49]:
response = client.delete(
    index=index_name,
    id=id
)
 
print('\nDeleting document:')
print_json(response)


Deleting document:
{
  "_index": "python-test-index",
  "_id": "1",
  "_version": 2,
  "result": "deleted",
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 1,
  "_primary_term": 1
}


In [50]:
response = client.indices.delete(index=index_name)
 
print('\nDeleting index:')
print_json(response)


Deleting index:
{
  "acknowledged": true
}


In [17]:
client.indices.get_alias(index = "*")

{'.plugins-ml-config': {'aliases': {}},
 '.opensearch-observability': {'aliases': {}},
 '.opensearch-sap-pre-packaged-rules-config': {'aliases': {}},
 '.opensearch-sap-log-types-config': {'aliases': {}},
 '.kibana_92668751_admin_1': {'aliases': {'.kibana_92668751_admin': {}}},
 'security-auditlog-2023.09.24': {'aliases': {}},
 'opensearch_dashboards_sample_data_flights': {'aliases': {}},
 'search_fun_test': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}},
 '.opendistro_security': {'aliases': {}}}

In [25]:
q = 'dog'
query = {
    'size': 5,
    'query': {
        'multi_match': {
            'query': q,
            'fields': ['body']
        }
    }
}

In [27]:
client.search(
    index = 'search_fun_test',
    body = query   
)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0902805,
  'hits': [{'_index': 'search_fun_test',
    '_id': 'doc_b',
    '_score': 1.0902805,
    '_source': {'title': 'Fox wins championship',
     'body': 'Wearing all red, the Fox jumped out to a lead in the race over the Dog.',
     'category': 'sports'}}]}}